In [1]:
import requests
import json
import pandas as pd

In [2]:
files_endpt = "https://api.gdc.cancer.gov/files"

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "file_name",
    #"analysis.workflow_type",
    #"experimental_strategy",
    "cases.project.primary_site",
    "cases.project.disease_type",
    "cases.diagnoses.primary_diagnosis",
    "cases.submitter_id",
    "cases.samples.portions.analytes.aliquots.submitter_id",
    "cases.diagnoses.tumor_stage",
    "cases.diagnoses.tumor_grade",
    "cases.diagnoses.progression_or_recurrence"
    "cases.diagnoses.vital_status",
    "cases.diagnoses.days_to_birth",
    "cases.diagnoses.days_to_death",
    "cases.diagnoses.morphology",
    "cases.diagnoses.tissue_or_organ_of_origin",
    "cases.samples.longest_dimension",
    "cases.samples.shortest_dimension",
    "cases.samples.pathology_report_uuid"
    #"cases.project.project_id"
]

fields = ','.join(fields)

In [6]:
#miRNA filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["miRNA Expression Quantification"]
            }
        },
          {
        "op": "in",
        "content":{
               "field": "files.experimental_strategy",
                "value": ["miRNA-Seq"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ['TCGA-BRCA']
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.program.name",
            "value": ["TCGA"]
            }
        }
        
    ]
}

In [7]:
# A POST is used, so the filter parameters can be passed directly as a Dict object.
##remove manifest to have file fields
params = {
   "return_type": "manifest",
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }

In [8]:
# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

#print(response.content.decode("utf-8"))

In [9]:
len(response.content.decode("utf-8"))

190733

In [10]:
with open("manifest.txt","w") as manifest:
    manifest.write(response.content.decode("utf-8"))

In [11]:
params = {
    "filters": json.dumps(filters),
    "fields": "primary_site,cases.project.project_id,file_name,cases.demographic.vital_status,cases.demographic.gender,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_last_follow_up,cases.demographic.days_to_death,cases.demographic.days_to_birth,cases.submitter_id,samples.portions.analytes.aliquots.submitter_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.tumor_stage,cases.exposures.years_smoked,cases.exposures.cigarettes_per_day,cases.samples.portions.analytes.aliquots.submitter_id,cases.samples.longest_dimension,cases.samples.shortest_dimension,cases.samples.tumor_descriptor",
    "format": "TSV",
    "size": "50000"
    }
response = requests.get("https://api.gdc.cancer.gov/files", headers = {"Content-Type": "application/json"}, params = params)
#print(response.content.decode("UTF-8"))
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))

In [12]:
df_files = pd.read_csv("files.txt", sep='\t')
df_files.drop("id", axis=1, inplace=True)
df_files.set_index("file_name", inplace=True)
df_files.head(2)

,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,cases.0.exposures.0.years_smoked,cases.0.project.project_id,cases.0.samples.0.longest_dimension,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cases.0.samples.0.shortest_dimension,cases.0.samples.0.tumor_descriptor,cases.0.submitter_id
file_name,,,,,,,,,,,,,,,,
d3042b0b-9211-48c5-a58f-d1669dd8acbb.mirbase21.mirnas.quantification.txt,-13138.0,3462.0,female,Dead,13138.0,NaN,not reported,stage iib,NaN,NaN,TCGA-BRCA,NaN,TCGA-BH-A1ES-01A-11R-A136-13,NaN,NaN,TCGA-BH-A1ES
1edab19b-3101-4766-a37f-cf4aca302dab.mirbase21.mirnas.quantification.txt,-22896.0,786.0,female,Dead,22896.0,656.0,not reported,stage iia,NaN,NaN,TCGA-BRCA,NaN,TCGA-E9-A1NG-11A-52R-A14L-13,NaN,NaN,TCGA-E9-A1NG


In [13]:
df_files["cases.0.samples.0.tumor_descriptor"].dropna().unique()

array([], dtype=float64)

In [14]:
df_files.to_csv("files_miRNA.dat", header=True)

## Files

In [15]:
# Tissue project filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene Expression Quantification"]
            }
        },
        {
        "op": "in",
        "content":{
               "field": "files.analysis.workflow_type",
                "value": ["HTSeq - FPKM"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ["TCGA-BRCA"]
            }
        }
        
    ]
}

In [16]:
params = {
    "filters": json.dumps(filters),
    "fields": "primary_site,cases.project.project_id,file_name,cases.demographic.vital_status,cases.demographic.gender,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_last_follow_up,cases.demographic.days_to_death,cases.demographic.days_to_birth,cases.submitter_id,samples.portions.analytes.aliquots.submitter_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.tumor_stage,cases.exposures.years_smoked,cases.exposures.cigarettes_per_day,cases.samples.portions.analytes.aliquots.submitter_id",
    "format": "TSV",
    "size": "50000"
    }
response = requests.get("https://api.gdc.cancer.gov/files", headers = {"Content-Type": "application/json"}, params = params)
#print(response.content.decode("UTF-8"))
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))

In [17]:
df_files = pd.read_csv("files.txt", sep='\t')
df_files.drop("id", axis=1, inplace=True)
df_files.set_index("file_name", inplace=True)
df_files.head(2)

,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,cases.0.exposures.0.years_smoked,cases.0.project.project_id,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cases.0.submitter_id
file_name,,,,,,,,,,,,,
da8cb4c3-4599-41d8-9f1e-6854d9996435.FPKM.txt.gz,-27977.0,NaN,female,Alive,27977.0,224.0,not reported,stage iia,NaN,NaN,TCGA-BRCA,TCGA-AN-A0AK-01A-21R-A00Z-07,TCGA-AN-A0AK
0978e2a7-73fc-4389-9e63-3d6fd42832e8.FPKM.txt.gz,-26274.0,NaN,female,Alive,26274.0,954.0,not reported,stage iiia,NaN,NaN,TCGA-BRCA,TCGA-A7-A26E-01B-06R-A277-07,TCGA-A7-A26E


In [18]:
df_files.to_csv("files_fpkm.dat", header=True)

# Merge

In [19]:
df_messangers = pd.read_csv("files_fpkm.dat")
df_messangers.head(2)

,file_name,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,cases.0.exposures.0.years_smoked,cases.0.project.project_id,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cases.0.submitter_id
0,da8cb4c3-4599-41d8-9f1e-6854d9996435.FPKM.txt.gz,-27977.0,NaN,female,Alive,27977.0,224.0,not reported,stage iia,NaN,NaN,TCGA-BRCA,TCGA-AN-A0AK-01A-21R-A00Z-07,TCGA-AN-A0AK
1,0978e2a7-73fc-4389-9e63-3d6fd42832e8.FPKM.txt.gz,-26274.0,NaN,female,Alive,26274.0,954.0,not reported,stage iiia,NaN,NaN,TCGA-BRCA,TCGA-A7-A26E-01B-06R-A277-07,TCGA-A7-A26E


In [20]:
df_miRNA = pd.read_csv("files_miRNA.dat")
df_miRNA.head(2)

,file_name,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,cases.0.exposures.0.years_smoked,cases.0.project.project_id,cases.0.samples.0.longest_dimension,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cases.0.samples.0.shortest_dimension,cases.0.samples.0.tumor_descriptor,cases.0.submitter_id
0,d3042b0b-9211-48c5-a58f-d1669dd8acbb.mirbase21...,-13138.0,3462.0,female,Dead,13138.0,NaN,not reported,stage iib,NaN,NaN,TCGA-BRCA,NaN,TCGA-BH-A1ES-01A-11R-A136-13,NaN,NaN,TCGA-BH-A1ES
1,1edab19b-3101-4766-a37f-cf4aca302dab.mirbase21...,-22896.0,786.0,female,Dead,22896.0,656.0,not reported,stage iia,NaN,NaN,TCGA-BRCA,NaN,TCGA-E9-A1NG-11A-52R-A14L-13,NaN,NaN,TCGA-E9-A1NG


In [21]:
df_messangers = df_messangers.loc[~df_messangers["cases.0.submitter_id"].duplicated(keep="first"),:].rename(columns={"file_name":"file_fpkm"})
df_miRNA = df_miRNA.loc[~df_miRNA["cases.0.submitter_id"].duplicated(keep="first"),:].rename(columns={"file_name":"file_miRNA"})

In [22]:
df_files = df_miRNA.set_index("cases.0.submitter_id").transpose().append(df_messangers.set_index("cases.0.submitter_id").transpose()).transpose()
df_files.head(2)

,file_miRNA,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,...,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,cases.0.exposures.0.years_smoked,cases.0.project.project_id,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id
cases.0.submitter_id,,,,,,,,,,,,,,,,,,,,,
TCGA-BH-A1ES,d3042b0b-9211-48c5-a58f-d1669dd8acbb.mirbase21...,-13138.0,3462.0,female,Dead,13138.0,NaN,not reported,stage iib,NaN,...,female,Dead,13138.0,NaN,not reported,stage iib,NaN,NaN,TCGA-BRCA,TCGA-BH-A1ES-06A-12R-A24H-07
TCGA-E9-A1NG,1edab19b-3101-4766-a37f-cf4aca302dab.mirbase21...,-22896.0,786.0,female,Dead,22896.0,656.0,not reported,stage iia,NaN,...,female,Dead,22896.0,656.0,not reported,stage iia,NaN,NaN,TCGA-BRCA,TCGA-E9-A1NG-11A-52R-A14M-07


In [23]:
df_files.to_csv("files_manifest.dat")

In [24]:
pd.read_csv("data_miRNA/002f73df-6c1d-4187-bab8-09908ed64a75/09a8fbb6-a236-4c64-be34-932db3fd3f06.mirbase21.mirnas.quantification.txt", sep="\t")["miRNA_ID"].to_csv("miRNA.txt", index=False, header=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data_miRNA/002f73df-6c1d-4187-bab8-09908ed64a75/09a8fbb6-a236-4c64-be34-932db3fd3f06.mirbase21.mirnas.quantification.txt'